In [2]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_autogluon
importlib.reload(preprocesamiento)
importlib.reload(model_autogluon)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


# Experimento 6: 
- Autogluon: Añade transformación de target
- Kaggle =  0.346


In [3]:
# df = model_autogluon.ensemble_de_ventanasValidacion()

In [4]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from scipy.stats import boxcox


# Carga y preparación de datos
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')

# Verificación y limpieza de datos
print("Verificando datos...")
print(f"Filas originales: {len(df)}")
df = df.dropna(subset=['periodo', 'product_id', 'tn'])
print(f"Filas después de limpieza: {len(df)}")

# Agregación y transformación
dfg = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
dfg['periodo_dt'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'tn': 'target', 'product_id':'item_id', 'periodo_dt': 'timestamp'}, inplace=True)
dfg.drop(columns=['periodo'], inplace=True)

# Filtrar productos
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]
print(f"Productos únicos a predecir: {len(dfg['item_id'].unique())}")

# Conversión a TimeSeriesDataFrame con verificación
if len(dfg) == 0:
    raise ValueError("El DataFrame está vacío después del filtrado")

try:
    data = TimeSeriesDataFrame.from_data_frame(
        dfg,
        id_column="item_id",
        timestamp_column="timestamp"
    )
    print("TimeSeriesDataFrame creado exitosamente")
    print(f"Número de series temporales: {len(data.item_ids)}")
except Exception as e:
    raise ValueError(f"Error al crear TimeSeriesDataFrame: {str(e)}")

all_predictions = []

for n_windows in range(1, 13):  # Probando con 2 ventanas
    print(f"\n--- Entrenamiento con {n_windows} ventana(s) ---")
    
    try:
        predictor = TimeSeriesPredictor(
            target='target',
            prediction_length=2,
            freq="M",
            eval_metric="MAPE"
        ).fit(
            data,
            num_val_windows=n_windows,
            verbosity=2  # Más detalle en logs
        )
        
        ##################
        # # Box-Cox transform para estabilizar varianza
        data['target'], lambda_param = boxcox(data['target'] + 1)  # +1 para valores positivos
        #####################################
        
        preds = predictor.predict(data)
        preds['mean'] = (preds['mean'] * lambda_param + 1) ** (1/lambda_param) ### # Inversión de la transformación


        print("Predicciones obtenidas exitosamente")
        
        # Procesamiento robusto de predicciones
        preds_202002 = preds.reset_index()
        preds_202002 = preds_202002[preds_202002['timestamp'] == '2020-02-29']
        
        if len(preds_202002) == 0:
            print(f"Advertencia: No hay predicciones para febrero 2020 con {n_windows} ventanas")
            continue
            
        preds_202002 = preds_202002[["item_id", "mean"]].rename(columns={
            "item_id": "product_id", 
            "mean": f"pred_windows_{n_windows}"
        })
        
        all_predictions.append(preds_202002.set_index("product_id"))
        print(f"Predicciones para {n_windows} ventanas procesadas")
        
    except Exception as e:
        print(f"Error durante el entrenamiento/predicción: {str(e)}")
        continue

# Verificación final antes de consolidar
if not all_predictions:
    raise ValueError("No se generaron predicciones válidas en ninguna iteración")

print("\nConsolidando resultados...")
final_df = pd.concat(all_predictions, axis=1)
print(f"DataFrame consolidado: {final_df.shape}")



# Resultado final
final_df = final_df.reset_index().sort_values('product_id')

# Guardado seguro
output_path = "./outputs/predicciones_exp_06v2_autogluon_v1.csv"
final_df.to_csv(output_path, index=False)
print(f"\nProceso completado exitosamente. Resultados guardados en: {output_path}")
print(f"Resumen de predicciones:\n{final_df.describe()}")



Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\AutogluonModels\ag-20250628_170134'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       2.42 GB / 15.89 GB (15.3%)
Disk Space Avail:   414.47 GB / 893.49 GB (46.4%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}



Verificando datos...
Filas originales: 31362
Filas después de limpieza: 31362
Productos únicos a predecir: 780
TimeSeriesDataFrame creado exitosamente
Número de series temporales: 780

--- Entrenamiento con 1 ventana(s) ---


train_data with frequency 'MS' has been resampled to frequency 'ME'.
Provided train_data has 22375 rows, 780 time series. Median time series length is 36 (min=4, max=36). 
	Removing 46 short time series from train_data. Only series with length >= 7 will be used for training.
	After filtering, train_data has 22132 rows, 734 time series. Median time series length is 36 (min=7, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-28 14:01:37
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'ChronosZeroShot[bolt_base]', 'ChronosFineTuned[bolt_small]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST', 'TiDE']
Training timeseries model Seaso

Predicciones obtenidas exitosamente
Predicciones para 1 ventanas procesadas

--- Entrenamiento con 2 ventana(s) ---


train_data with frequency 'MS' has been resampled to frequency 'ME'.
Provided train_data has 22375 rows, 780 time series. Median time series length is 36 (min=4, max=36). 
	Removing 75 short time series from train_data. Only series with length >= 9 will be used for training.
	After filtering, train_data has 21916 rows, 705 time series. Median time series length is 36 (min=9, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-28 14:20:27
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'ChronosZeroShot[bolt_base]', 'ChronosFineTuned[bolt_small]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST', 'TiDE']
Training timeseries model Seaso

...

Predicciones obtenidas exitosamente
Predicciones para 10 ventanas procesadas

--- Entrenamiento con 11 ventana(s) ---


train_data with frequency 'MS' has been resampled to frequency 'ME'.
Provided train_data has 22375 rows, 780 time series. Median time series length is 36 (min=4, max=36). 
	Removing 231 short time series from train_data. Only series with length >= 27 will be used for training.
	After filtering, train_data has 19433 rows, 549 time series. Median time series length is 36 (min=27, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-29 10:55:50
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'ChronosZeroShot[bolt_base]', 'ChronosFineTuned[bolt_small]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST', 'TiDE']
Training timeseries model Se

Predicciones obtenidas exitosamente
Predicciones para 11 ventanas procesadas

--- Entrenamiento con 12 ventana(s) ---


train_data with frequency 'MS' has been resampled to frequency 'ME'.
Provided train_data has 22375 rows, 780 time series. Median time series length is 36 (min=4, max=36). 
	Removing 250 short time series from train_data. Only series with length >= 29 will be used for training.
	After filtering, train_data has 18914 rows, 530 time series. Median time series length is 36 (min=29, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-29 15:05:09
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'ChronosZeroShot[bolt_base]', 'ChronosFineTuned[bolt_small]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST', 'TiDE']
Training timeseries model Se

Predicciones obtenidas exitosamente
Predicciones para 12 ventanas procesadas

Consolidando resultados...
DataFrame consolidado: (780, 12)

Proceso completado exitosamente. Resultados guardados en: ./outputs/predicciones_exp_06v2_autogluon_v1.csv
Resumen de predicciones:
         product_id  pred_windows_1  pred_windows_2  pred_windows_3  \
count    780.000000      780.000000      780.000000      780.000000   
mean   20541.421795       44.386217        2.864517        2.619990   
std      353.984342      132.396857        1.091577        0.849225   
min    20001.000000        1.020477        1.006280        1.009172   
25%    20238.750000        2.792217        1.957882        1.944129   
50%    20511.500000        9.190886        2.906400        2.679356   
75%    20818.500000       24.726772        3.586744        3.205717   
max    21276.000000     1948.509096        5.790449        4.706129   

       pred_windows_4  pred_windows_5  pred_windows_6  pred_windows_7  \
count      780.0

In [ ]:
# final_df = final_df.reset_index()
output_path = "./outputs/predicciones_exp_06v2_autogluon_v1_original.csv"
final_df.to_csv(output_path, index=False)

In [5]:
# 1. Definir pesos relativos
weights = pd.Series({
    'pred_windows_1': 1,    # 1 ventana → peso 1
    'pred_windows_2': 2,     # 2 ventanas → peso 2
    'pred_windows_3': 3,     # 3 ventanas → peso 3
    'pred_windows_4': 4,     # 4 ventanas → peso 4
    'pred_windows_5': 5,     # 5 ventanas → peso 5
    'pred_windows_6': 6,     # 6 ventanas → peso 6
    'pred_windows_7': 7,     # 7 ventanas → peso 7
    'pred_windows_8': 8,     # 8 ventanas → peso 8
    'pred_windows_9': 9,     # 9 ventanas → peso 9
    'pred_windows_10': 10,   # 10 ventanas → peso 10
    'pred_windows_11': 11,   # 11 ventanas → peso 11
    'pred_windows_12': 12   # 12 ventanas → peso 12
})

# 2. Para cada producto:
for product_id, row in final_df.iterrows():
    pred_1 = row['pred_windows_1']
    pred_2 = row['pred_windows_2']
    pred_3 = row['pred_windows_3']
    pred_4 = row['pred_windows_4']
    pred_5 = row['pred_windows_5']
    pred_6 = row['pred_windows_6']
    pred_7 = row['pred_windows_7']
    pred_8 = row['pred_windows_8']
    pred_9 = row['pred_windows_9']
    pred_10 = row['pred_windows_10']
    pred_11 = row['pred_windows_11']
    pred_12 = row['pred_windows_12']
    
    # 3. Aplicar ponderación
    total_ponderado = (pred_1 * weights['pred_windows_1']) + (pred_2 * weights['pred_windows_2']) + \
                      (pred_3 * weights['pred_windows_3']) + (pred_4 * weights['pred_windows_4']) + \
                      (pred_5 * weights['pred_windows_5']) + (pred_6 * weights['pred_windows_6']) + \
                      (pred_7 * weights['pred_windows_7']) + (pred_8 * weights['pred_windows_8']) + \
                      (pred_9 * weights['pred_windows_9']) + (pred_10 * weights['pred_windows_10']) + \
                      (pred_11 * weights['pred_windows_11']) + (pred_12 * weights['pred_windows_12'])
                      
    suma_pesos = weights.sum()  # 1 + 2 = 3
    
    # 4. Guardar resultado
    final_df.loc[product_id, 'pred_ponderado'] = total_ponderado / suma_pesos

In [6]:
final_df = final_df.reset_index()
output_path = "./outputs/predicciones_exp_06v2_autogluon_v1_ponderado.csv"
final_df.to_csv(output_path, index=False)

In [7]:
final_df

,index,product_id,pred_windows_1,pred_windows_2,pred_windows_3,pred_windows_4,pred_windows_5,pred_windows_6,pred_windows_7,pred_windows_8,pred_windows_9,pred_windows_10,pred_windows_11,pred_windows_12,pred_ponderado
0,0,20001,1948.509096,5.790449,4.706129,4.831068,4.825094,4.822133,4.819272,4.800056,4.815587,4.824542,4.811994,4.821208,29.757672
1,1,20002,1558.915266,5.684631,4.622296,4.765412,4.761919,4.761888,4.755986,4.728475,4.733178,4.755697,4.748613,4.752559,24.694035
2,2,20003,979.181161,5.551966,4.544408,4.670640,4.671748,4.657599,4.649639,4.626124,4.653688,4.669213,4.647797,4.654296,17.166850
3,3,20004,787.115949,5.388176,4.450024,4.571076,4.568240,4.550122,4.539017,4.545650,4.557924,4.558587,4.558112,4.563314,14.606578
4,4,20005,826.450558,5.373068,4.439067,4.559301,4.547415,4.534181,4.524501,4.538179,4.573684,4.538836,4.542762,4.551798,15.100175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,645,21263,1.024395,1.016406,1.020688,1.010437,1.010320,0.995073,0.978277,1.020568,1.010700,1.010829,1.014955,1.013705,1.009347
776,671,21265,1.074313,1.049816,1.048587,1.047709,1.043388,1.047325,1.046163,1.066336,1.059532,1.049945,1.054352,1.056455,1.053536
777,672,21266,1.076876,1.052437,1.050657,1.052725,1.049885,1.052676,1.051051,1.069460,1.065019,1.055284,1.058027,1.061343,1.058148
778,673,21267,1.058716,1.020841,1.034657,1.037764,1.037764,1.039215,1.038616,1.053851,1.055016,1.041907,1.048799,1.048106,1.044986


In [8]:
final_df_copy = final_df.copy()

final_df_copy = final_df_copy[['product_id', 'pred_windows_1']].rename(columns={'pred_windows_1': 'tn'})

output_path = "./outputs/predicciones_exp_06v2_autogluon_v1_windows_1.csv"

final_df_copy.to_csv(output_path, sep=',', index=False)